# MENA Analysis: Road Density
## OpenStreetMap 2023

### Load packages and setup

In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np

In [2]:
# Define directories
dir_shp = '/Users/ashitakarl/Library/CloudStorage/Dropbox/WB/MENA_WorldBank/Boundaries/'
dir_in = '/Users/ashitakarl/Desktop/Project/WB/MENA/OSM/'
dir_out = '/Users/ashitakarl/Library/CloudStorage/Dropbox/WB/MENA_WorldBank/Index/'

# Define output FILE
VAR = 'ROAD'
FILE = 'ADM2_' + VAR + '.csv'

In [3]:
%matplotlib inline

### Load shapefile

In [4]:
# Load shapefile
MENA_shp = gpd.read_file(dir_shp + 'MENA_ADM2.shp')
MENA_shp['area_sqkm'] = MENA_shp['geometry'].area

MENA_shp.crs

/var/folders/xr/g_krxdrx6zq93vshtvkz5hym0000gn/T/ipykernel_13923/621451912.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  MENA_shp['area_sqkm'] = MENA_shp['geometry'].area


<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

### Load all the OSM road layers

In [5]:
# Country list
MENA_list = ['algeria', 'djibouti', 'egypt', 'gcc-states', 'iran', 'iraq', 'israel-and-palestine',
            'jordan', 'lebanon', 'libya', 'malta', 'morocco', 'syria', 'tunisia', 'yemen']
MENA_list = MENA_list[1::]

# Road type list
type_list = 'footway', 'pedestrian', 'path', 'steps', 'track_grade4', 'track_grade5', 'unknown', 'cycleway', 'bridleway'

In [6]:
osm_road_full = gpd.read_file(dir_in + 'algeria-latest-free/gis_osm_roads_free_1.shp').to_crs('EPSG:4326')
osm_road_full = osm_road_full[~osm_road_full['fclass'].isin(type_list)] 

In [7]:
for COUNTRY in MENA_list:
    osm_road_temp = gpd.read_file(dir_in + COUNTRY + '-latest-free/gis_osm_roads_free_1.shp').to_crs('EPSG:4326')
    osm_road_temp = osm_road_temp[~osm_road_temp['fclass'].isin(type_list)] 
    osm_road_full = gpd.pd.concat([osm_road_full, osm_road_temp])
    print('DONE: ' + COUNTRY + ' :)')

DONE: djibouti :)
DONE: egypt :)
DONE: gcc-states :)
DONE: iran :)
DONE: iraq :)
DONE: israel-and-palestine :)
DONE: jordan :)
DONE: lebanon :)
DONE: libya :)
DONE: malta :)
DONE: morocco :)
DONE: syria :)
DONE: tunisia :)
DONE: yemen :)


### Calculate road length and density

In [8]:
# Intersect the roads (lines) with boundaries (polygons)
road_intersect = gpd.sjoin(MENA_shp, osm_road_full)

In [9]:
# Calculate the length of each segment of roads
road_intersect['length_km'] = road_intersect['geometry'].length

/var/folders/xr/g_krxdrx6zq93vshtvkz5hym0000gn/T/ipykernel_13923/2992236208.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'length' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  road_intersect['length_km'] = road_intersect['geometry'].length


In [10]:
road_intersect.length_km.head()

0     1.103580
7     7.474568
19    3.884721
0     1.103580
0     1.103580
Name: length_km, dtype: float64

In [11]:
# Summarize by ADM
road_sum = road_intersect.groupby('ID_ADM').sum('length_km')
road_sum.reset_index(inplace=True)

# Create index
road_sum['road_den'] = road_sum.length_km / road_sum.area_sqkm
road_sum = road_sum[['ID_ADM', 'road_den']]

In [12]:
road_sum

,ID_ADM,road_den
0,5748,119.778218
1,5749,152.296125
2,5750,193.705651
3,5751,442.920875
4,5752,139.478071
...,...,...
2925,65742,20.026486
2926,65743,20.780945
2927,65744,2.634185
2928,65745,22.244970


In [14]:
road_sum.to_csv(dir_out + FILE, index = False)